## Bibliotecas

In [1]:
import numpy as np
from scipy.special import psi, polygamma
from scipy.stats import beta

## MÉTODO

### Newton_Raphson

In [76]:
def grad(theta, x):
    alpha = theta[0]
    beta = theta[1]
    n = len(x)

    h1 = psi(alpha) - psi(alpha + beta) - (1/n)*sum(np.log(x))
    h2 = psi(beta) - psi(alpha + beta) - (1/n)*sum(np.log(1-x))
    return[h1,h2]

def inv_Hessian(theta, x):
    alpha = theta[0]
    beta = theta[1]

    # definir matriz
    h11 = polygamma(1, alpha) - polygamma(1, alpha + beta)
    h12 = -polygamma(1, alpha + beta)
    h21 = -polygamma(1, alpha + beta)
    h22 = polygamma(1, beta) - polygamma(1, alpha + beta)
    H_inv = np.linalg.inv([[h11, h12],[h21, h22]])
    return H_inv
    
def ML_beta(x, niter):
    # chute inicial vetor theta0 = (alpha0, beta0)
    theta0 = np.array([0.1, 0.1])
    # criar lista para armazenar todos pares de estimativas de parâmetros
    thetas = [theta0]

    for k in range(1, niter+1):
        theta_k = thetas[k-1] - np.dot( inv_Hessian(thetas[k-1],x), grad(thetas[k-1],x) )
        thetas.append(theta_k)
    return thetas


### Rodando

In [77]:
np.random.seed(512)
n = 1000
N = 5000
alpha_dist = 3
beta_dist = 5

sample_parameters = []

for _ in range(N):
    x = beta.rvs(alpha_dist, beta_dist, size=n)
    estimated_parameters = ML_beta(x, niter=10)[-1] # Taking the last two estimated parameters from the 10 steps (10 different values of pair of alpha and beta from the method)
    sample_parameters.append(estimated_parameters)

# Display the estimated parameters
#print("Estimated parameters for", N, "samples:")
#for i, params in enumerate(sample_parameters[:]): # Displaying parameters for the the : samples
    #print(f"Sample {i+1}: alpha={params[0]}, beta={params[1]}")



In [78]:
array1 = np.array(sample_parameters)  

estimated_bias_alpha = np.mean(array1[:,0]) - alpha_dist
estimated_bias_beta = np.mean(array1[:,1]) - beta_dist

print(estimated_bias_alpha)
print(estimated_bias_beta)


estimated_eqm_alpha = np.mean( (array1[:,0] - alpha_dist)**2 )
estimated_eqm_beta = np.mean( (array1[:,1] - beta_dist)**2 ) 

print(estimated_eqm_alpha)
print(estimated_eqm_beta)

0.007852477622572351
0.01089015107255431
0.016104817404162645
0.0471181529039052


### bias and eqm as functions

In [79]:
array1 = np.array(sample_parameters)  

def bias(theta, array1):
    alpha = theta[0]
    beta = theta[1]
    bias_alpha = np.mean(array1[:,0]) - alpha
    bias_beta = np.mean(array1[:,1]) - beta
    return bias_alpha, bias_beta

def EQM(theta, arrray1):
    alpha = theta[0]
    beta = theta[1]
    eqm_alpha = np.mean( (array1[:,0] - alpha)**2 )
    eqm_beta = np.mean( (array1[:,1] - beta)**2 )
    return eqm_alpha, eqm_beta

print(bias([3,5],array1))
print(EQM([3,5],array1))

(0.007852477622572351, 0.01089015107255431)
(0.016104817404162645, 0.0471181529039052)
